# Random Forest 3

En este tercer notebook hacemos random forest con Cross Validation y gridsearch para practicar la implementación.

In [ ]:
# paquetes
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
# Repetimos los pasos del Random_Forest_1

# importamos los datos
df_import = pd.read_csv('df_datos.csv', index_col='Unnamed: 0')

# definimos las variables predictivas (X) y la categoría (y)
df_X = df_import.drop(columns = ['id_ecg','categoria','patient_id'])
ps_y = df_import['categoria'] # ps = pandas series

# label encoder (codificar categorías)
label_encoder = LabelEncoder()
ps_y_encoded = label_encoder.fit_transform(ps_y)

# vamos a dividir los datos para el grupo de entrenamiento y el de prueba.
X_train, X_test, y_train, y_test = train_test_split(df_X,
                                                    ps_y_encoded,
                                                    train_size= 2000, # diferencia: fijamos el número de elementos
                                                    random_state=10)

# vamos a imprimir los resultados del splitting
train_size = X_train.shape[0]
test_size = X_test.shape[0]
porcentaje_train = train_size/(train_size+test_size)
porcentaje_test = test_size/(train_size+test_size)
print()
print(f'\033[1m tenemos {train_size:d} eventos para el training ({porcentaje_train:.2%}). \033[0m') # f-strings y negritas
print()
print(f'\033[1m tenemos {test_size:d} eventos para el testing ({porcentaje_test:.2%}). \033[0m') # f-strings y negritas
print()

In [ ]:
# creamos el modelo abstracto de gridsearch con cross validation

# definimos el random forest sin los parámetros a mover
rf_abstracto = RandomForestClassifier(criterion='gini',
                                      max_depth=None,
                                      min_samples_split=10,
                                      bootstrap=True,
                                      random_state=10)

# método de foldeo
cross_validation_strategy = KFold(n_splits=10,
                                  shuffle=True,
                                  random_state=10)

#gridsearch: ejecutar CrossValidation con diferentes parámetros.
gridsearch_abstracto = GridSearchCV(estimator=rf_abstracto, # modelo a evaluar
                                    param_grid={'n_estimators':[15,20,25],  # hiperparámetro a modificar 1: número de árboles
                                                'max_features':[5,6,7]},  # hiperparámetro a modificar 2: número de variables para la cosntrucción de árboles
                                    scoring='accuracy', # evaluación
                                    cv=cross_validation_strategy, # cross validation
                                    error_score='raise') # que nos avise si hay errores

In [ ]:
# ejecutamos gridsearch con las bases de datos
gridsearch = gridsearch_abstracto.fit(X_train, y_train)

In [ ]:
# vamos a visualizar los resultados

# guardamos los resultados que nos interesan
gridsearch_resultados = gridsearch.cv_results_ # aquí están todos los resultados resumidos
parametros = gridsearch_resultados['params'] # guardamos los parámetros y...
resultados = gridsearch_resultados['mean_test_score'] # el resultado del modelo con esos parámetros.

# guardamos el i-esimo resultado con sus parámetros
for i, dic in enumerate(parametros):
    dic.update({'accuracy':resultados[i]})

# vemos cuales parametros tenemos
valores_n_estimators = np.unique([p['n_estimators'] for p in parametros])
valores_max_features = np.unique([p['max_features'] for p in parametros])

# creamos un DataFrame de ceros
df_resultados = pd.DataFrame(data=np.zeros((len(valores_n_estimators),len(valores_max_features))),
                             index=valores_n_estimators,
                             columns=valores_max_features)

# llenamos el dataframe
for i, n_estimator in enumerate(valores_n_estimators):
    for j, max_feature in enumerate(valores_max_features):
        df_resultados.iloc[i,j] = [x['accuracy'] for x in parametros if (x['n_estimators']==n_estimator) and (x['max_features']==max_feature)][0]        

# graficamos
ax, fig = plt.subplots()
ax = plt.imshow(df_resultados, vmin=0.4, vmax=0.6, cmap='Blues')
plt.colorbar()
plt.xticks(ticks=np.arange(len(df_resultados.columns)), labels=df_resultados.columns)
plt.yticks(ticks=np.arange(len(df_resultados.index)), labels=df_resultados.index)
plt.ylabel("Número de árboles")
plt.xlabel("Número de variables")
plt.title("Resultados de gridsearch")
plt.plot()